In [ ]:
! pip install rdkit
! pip install torch_geometric
! pip install transformers
! pip install biopython
! pip install numpy
! pip install suds

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 961.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 7.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from os.path import join
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data, DataLoader
from rdkit import Chem
import rdkit
from rdkit.Chem import AllChem
from Bio.PDB import PDBList, PDBParser
import numpy as np
import requests
from sklearn.metrics import r2_score

# Define your inchi_to_graph function here
# and any other necessary functions or imports


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_train = pd.read_pickle(join("/content", "drive", "MyDrive", "data", "kcat_data", "splits", "train_df_kcat.pkl"))
data_test = pd.read_pickle(join("/content", "drive", "MyDrive", "data", "kcat_data", "splits", "test_df_kcat.pkl"))


data_train.rename(columns = {"geomean_kcat" :"log10_kcat"}, inplace = True)
data_test.rename(columns = {"geomean_kcat" :"log10_kcat"}, inplace = True)

data_train.keys()

Index(['Reaction ID', 'Sequence ID', 'kcat_values', 'Uniprot IDs',
       'from_BRENDA', 'from_Sabio', 'from_Uniprot', 'checked', 'Sequence',
       'substrates', 'products', 'MW_frac', 'max_kcat_for_UID',
       'max_kcat_for_RID', 'ECs', 'max_kcat_for_EC', 'structural_fp',
       'difference_fp', 'ESM1b', 'ESM1b_ts', 'log10_kcat', 'frac_of_max_UID',
       'frac_of_max_RID', 'frac_of_max_EC'],
      dtype='object')

In [ ]:
data_train = data_train[['substrates', 'products', 'Uniprot IDs', 'difference_fp', 'ESM1b', 'log10_kcat', 'difference_fp']]
data_test = data_test[['substrates', 'products', 'Uniprot IDs', 'difference_fp', 'ESM1b', 'log10_kcat', 'difference_fp']]


# Molecule to Graph Stucture

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
import torch
from torch_geometric.data import Data

def inchi_to_graph(inchi):
    try:
        mol = Chem.MolFromInchi(inchi)
        if mol is None:
            raise ValueError("Invalid InChI string")

        mol = Chem.AddHs(mol)  # Ensure hydrogen atoms are included
        AllChem.EmbedMolecule(mol)
        AllChem.UFFOptimizeMolecule(mol)

        # Create node features
        atom_features = []
        for atom in mol.GetAtoms():
            atom_features.append([
                atom.GetAtomicNum(),  # Atomic number
                atom.GetTotalDegree(),  # Degree
                atom.GetFormalCharge(),  # Formal charge
                atom.GetNumImplicitHs(),  # Number of implicit hydrogens
                atom.GetIsAromatic(),  # Aromaticity
                atom.GetHybridization().real  # Hybridization
            ])

        # Create edge indices and edge attributes
        edge_index = []
        edge_attr = []
        for bond in mol.GetBonds():
            i = bond.GetBeginAtomIdx()
            j = bond.GetEndAtomIdx()
            edge_index.append([i, j])
            edge_index.append([j, i])  # Undirected graph
            edge_attr.append([
                bond.GetBondTypeAsDouble(),  # Bond type
                bond.GetIsConjugated(),  # Conjugation
                bond.IsInRing()  # Ring status
            ])
            edge_attr.append([
                bond.GetBondTypeAsDouble(),  # Bond type
                bond.GetIsConjugated(),  # Conjugation
                bond.IsInRing()  # Ring status
            ])

        edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
        atom_features = torch.tensor(atom_features, dtype=torch.float)
        edge_attr = torch.tensor(edge_attr, dtype=torch.float)

        return Data(x=atom_features, edge_index=edge_index, edge_attr=edge_attr)

    except ValueError as ve:
        print(f"ValueError: {ve}")
    except RuntimeError as re:
        print(f"RuntimeError: {re}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# Example usage
inchi_str = "InChI=1S/CH4/h1H4"  # Methane
graph_data = inchi_to_graph(inchi_str)
print(graph_data)


Data(x=[5, 6], edge_index=[2, 8], edge_attr=[8, 3])


In [ ]:
import torch
from torch_geometric.data import Data

def aggregate_graphs(graphs):
    # Combine node features
    combined_x = torch.cat([g.x for g in graphs], dim=0)

    # Combine edge indices and add offsets to account for the combined node indices
    edge_index_list = []
    edge_attr_list = []
    offset = 0
    for g in graphs:
        edge_index_list.append(g.edge_index + offset)
        if 'edge_attr' in g:
            edge_attr_list.append(g.edge_attr)
        offset += g.num_nodes
    combined_edge_index = torch.cat(edge_index_list, dim=1)
    combined_edge_attr = torch.cat(edge_attr_list, dim=0) if edge_attr_list else None

    return Data(x=combined_x, edge_index=combined_edge_index, edge_attr=combined_edge_attr)

# Example usage with multiple InChIs
inchi_list = ["InChI=1S/CH4/h1H4", "InChI=1S/H2O/h1H2"]
graphs = [inchi_to_graph(inchi) for inchi in inchi_list]
aggregated_graph = aggregate_graphs(graphs)
print(aggregated_graph)


Data(x=[8, 6], edge_index=[2, 12], edge_attr=[12, 3])


# Protein to Graph Structure

In [ ]:

# Function to map UniProt ID to PDB IDs
def map_uniprot_to_pdb(uniprot_id):
    url = "https://rest.uniprot.org/idmapping/run"
    data = {
        'from': 'UniProtKB_AC-ID',
        'to': 'PDB',
        'ids': uniprot_id
    }
    response = requests.post(url, data=data)
    job_id = response.json()['jobId']

    # Check the status of the job
    status_url = f"https://rest.uniprot.org/idmapping/status/{job_id}"

    status_response = requests.get(status_url)
    status = status_response.json()
    try:
          # Fetch the results
      results_url = f"https://rest.uniprot.org/idmapping/results/{job_id}"
      results_response = requests.get(results_url)
      results = results_response.json()
      pdb_ids = [item['to'] for item in results['results']]
      return pdb_ids

    except Exception as e:
      return ['none']




# Function to fetch protein sequence from UniProt
def fetch_protein_sequence(uniprot_id):
    url = f'https://www.uniprot.org/uniprot/{uniprot_id}.fasta'
    response = requests.get(url)
    fasta_data = response.text
    return ''.join(fasta_data.split('\n')[1:])




In [ ]:
import torch
import networkx as nx
import numpy as np
from Bio.PDB import PDBParser, PDBList, NeighborSearch
from torch_geometric.data import Data
from scipy.spatial import KDTree
import logging

logging.basicConfig(level=logging.INFO)

def fetch_protein_structure(pdb_id):
    try:
        pdbl = PDBList()
        pdb_file = pdbl.retrieve_pdb_file(pdb_id, file_format="pdb", pdir='/tmp')
        parser = PDBParser(QUIET=True)
        structure = parser.get_structure(pdb_id, pdb_file)
        return structure
    except Exception as e:
        logging.error(f"Error fetching structure for PDB ID {pdb_id}: {e}")
        return None

def residue_to_feature(residue):
    # Converting residue to feature vector
    residue_types = ['ALA', 'CYS', 'ASP', 'GLU', 'PHE', 'GLY', 'HIS', 'ILE', 'LYS', 'LEU', 'MET', 'ASN', 'PRO', 'GLN', 'ARG', 'SER', 'THR', 'VAL', 'TRP', 'TYR']
    charge = {'ALA': 0, 'CYS': 0, 'ASP': -1, 'GLU': -1, 'PHE': 0, 'GLY': 0, 'HIS': 0.1, 'ILE': 0, 'LYS': 1, 'LEU': 0, 'MET': 0, 'ASN': 0, 'PRO': 0, 'GLN': 0, 'ARG': 1, 'SER': 0, 'THR': 0, 'VAL': 0, 'TRP': 0, 'TYR': 0}

    hydrophobicity = {'ALA': 1.8, 'CYS': 2.5, 'ASP': -3.5, 'GLU': -3.5, 'PHE': 2.8, 'GLY': -0.4, 'HIS': -3.2, 'ILE': 4.5, 'LYS': -3.9, 'LEU': 3.8, 'MET': 1.9, 'ASN': -3.5, 'PRO': -1.6, 'GLN': -3.5, 'ARG': -4.5, 'SER': -0.8, 'THR': -0.7, 'VAL': 4.2, 'TRP': -0.9, 'TYR': -1.3}
    polarity = {'ALA': 8.1, 'CYS': 5.5, 'ASP': 13.0, 'GLU': 12.3, 'PHE': 5.2, 'GLY': 9.0, 'HIS': 10.4, 'ILE': 5.2, 'LYS': 11.3, 'LEU': 4.9, 'MET': 5.7, 'ASN': 11.6, 'PRO': 8.0, 'GLN': 10.5, 'ARG': 10.5, 'SER': 9.2, 'THR': 8.6, 'VAL': 5.9, 'TRP': 5.4, 'TYR': 6.2}


    feature = []
    residue_type_one_hot = [1 if residue.get_resname() == res_type else 0 for res_type in residue_types]
    feature.extend(residue_type_one_hot)
    feature.append(weight[residue.get_resname()])
    feature.append(charge[residue.get_resname()])
    ca_atom = residue['CA']
    feature.extend(ca_atom.coord.tolist())
    return feature

def protein_structure_to_graph(structure, distance_threshold=10.0):
    graph = nx.Graph()
    residue_idx = {}
    idx = 0

    try:
        for model in structure:
            for chain in model:
                for residue in chain:
                    if 'CA' in residue:
                        graph.add_node(idx, residue=residue, feature=residue_to_feature(residue))
                        residue_idx[residue] = idx
                        idx += 1

        ca_atoms = [residue['CA'] for residue in residue_idx.keys()]
        coordinates = [atom.coord for atom in ca_atoms]
        kdtree = KDTree(coordinates)

        for i, coord in enumerate(coordinates):
            indices = kdtree.query_ball_point(coord, distance_threshold)
            for j in indices:
                if i != j:
                    distance = np.linalg.norm(coord - coordinates[j])
                    graph.add_edge(i, j, weight=distance)

    except Exception as e:
        logging.error(f"Error creating residue graph: {e}")

    return graph_to_pyg_data(graph)

def create_default_protein_graph():
    """Create a default protein graph with predefined nodes and edges."""
    graph = nx.Graph()
    # Define a few nodes with arbitrary features
    default_features = [
        [1] * 20 + [1.8, 8.1, 0, 0, 0, 0],  # Node 0 (ALA)
        [0] * 20 + [2.5, 5.5, 0, 1, 1, 1],  # Node 1 (CYS)
        [0] * 20 + [-3.5, 13.0, -1, 2, 2, 2],  # Node 2 (ASP)
        [0] * 20 + [4.5, 5.2, 0, 3, 3, 3]  # Node 3 (ILE)
    ]
    for idx, features in enumerate(default_features):
        graph.add_node(idx, feature=features)

    # Add some edges with arbitrary distances
    graph.add_edge(0, 1, weight=4.0)
    graph.add_edge(1, 2, weight=5.0)
    graph.add_edge(2, 3, weight=6.0)
    graph.add_edge(3, 0, weight=4.5)

    return graph_to_pyg_data(graph)

def graph_to_pyg_data(graph):
    try:
        node_features = [data['feature'] for _, data in graph.nodes(data=True)]
        x = torch.tensor(node_features, dtype=torch.float)

        edge_index = []
        edge_attr = []
        for edge in graph.edges(data=True):
            edge_index.append([edge[0], edge[1]])
            edge_index.append([edge[1], edge[0]])
            edge_attr.append([edge[2]['weight']])
            edge_attr.append([edge[2]['weight']])

        edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
        edge_attr = torch.tensor(edge_attr, dtype=torch.float)

        data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
        return data

    except Exception as e:
        logging.error(f"Error converting graph to PyG data: {e}")
        return None

# Example usage
pdb_id = "1A8M"  # Example PDB ID
structure = fetch_protein_structure(pdb_id)
if structure:
    graph = protein_structure_to_graph(structure)
else:
    logging.info(f"PDB ID {pdb_id} not found. Using default protein graph.")
    graph = create_default_protein_graph()

print(graph)


Residue:  ARG Feature:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, -4.5, 10.5, 1, 32.11399841308594, 47.72800064086914, 65.34300231933594]
Residue:  THR Feature:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, -0.7, 8.6, 0, 32.39099884033203, 46.75699996948242, 61.64899826049805]
Residue:  PRO Feature:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, -1.6, 8.0, 0, 32.42499923706055, 43.24700164794922, 60.00899887084961]
Residue:  SER Feature:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, -0.8, 9.2, 0, 34.37200164794922, 41.25, 57.426998138427734]
Residue:  ASP Feature:  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -3.5, 13.0, -1, 38.013999938964844, 40.9010009765625, 56.49599838256836]
Residue:  LYS Feature:  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -3.9, 11.3, 1, 37.45800018310547, 42.58100128173828, 53.0880012512207]
Residue:  PRO Feature:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,

# Data Engineering

In [ ]:
final_data = []
for item in processed_data:
  if item['aggregated_substrate_graph'] is not None and item['aggregated_product_graph'] is not None and item['protein_graph'] is not None:
    final_data.append(item)

In [ ]:
cleaned_data = []
for item in final_data:
  combined_graph = aggregate_graphs([item['aggregated_substrate_graph'], item['aggregated_product_graph']])
  cleaned_data.append({
      'combined': combined_graph,
      'protein_graph': item['protein_graph'],
      'log10_kcat': item['log10_kcat']
  })

In [ ]:
def make_datasets(x, y):

  # Initialize an empty list to hold the processed data
  processed_data = []

  # Process each row in the DataFrame
  for index, row in data_test.iterrows():
      if index > y:
        break
      if index >= x:

        # Process substrates
        print("Index: ", index ," Out of ", len(data_test))
        substrate_graphs = [inchi_to_graph(inchi) for inchi in row['substrates']]
        for item in substrate_graphs:
          if item is None:
            substrate_graphs.remove(item)
        if substrate_graphs:
            aggregated_substrate_graph = aggregate_graphs(substrate_graphs)
        else:
            aggregated_substrate_graph = None

        # Process products
        product_graphs = [inchi_to_graph(inchi) for inchi in row['products']]
        for item in product_graphs:
          if item is None:
            product_graphs.remove(item)
        if product_graphs and substrate_graphs:
          aggregated_product_graph = aggregate_graphs(product_graphs)
        else:
          aggregated_product_graph = None

        # Process protein substrate
        uniprot_id = row['Uniprot IDs']

        pdb_ids = map_uniprot_to_pdb(uniprot_id)
        if pdb_ids:
            pdb_id = pdb_ids[0]
            protein_structure = fetch_protein_structure(pdb_id)
            protein_graph = protein_structure_to_graph(protein_structure)

        else:
            protein_graph = None


        # Ground truth label
        log10_kcat = row['log10_kcat']
        embedding = row['ESM1b']
        fingerprint = row['difference_fp']
        # Append the processed data to the list
        processed_data.append({
            'aggregated_substrate_graph': aggregated_substrate_graph,
            'aggregated_product_graph': aggregated_product_graph,
            'protein_graph': protein_graph,
            'esm_embedding': embedding,
            'log10_kcat': log10_kcat,
            'fingerprint': fingerprint
        })
  # Define the path to save the processed data
  file_path = '/content/drive/My Drive/kcat_minidatasets/testNew' + str(y) + '.pkl'

  # Save the list to the specified file
  with open(file_path, 'wb') as file:
      pickle.dump(processed_data, file)

  print(f"List saved to {file_path}")

In [ ]:
make_datasets(0, 200)
make_datasets(200, 400)
make_datasets(400, 600)
make_datasets(600, 800)
make_datasets(800, 849)



Index:  0  Out of  850
Structure exists: '/tmp/pdb2yor.ent' 
Index:  1  Out of  850
Structure exists: '/tmp/pdb6aym.ent' 
Index:  2  Out of  850


[11:34:41] WARNING: not removing hydrogen atom without neighbors
[11:34:41] WARNING: not removing hydrogen atom without neighbors


Index:  3  Out of  850


[11:34:43] WARNING: not removing hydrogen atom without neighbors
[11:34:43] WARNING: not removing hydrogen atom without neighbors


Structure exists: '/tmp/pdb6n2h.ent' 


...

Index:  846  Out of  850
Index:  847  Out of  850
Index:  848  Out of  850


ERROR:root:Error creating residue graph: 'MSE'
[12:06:58] UFFTYPER: Unrecognized atom type: Fe5+2 (0)
[12:06:58] UFFTYPER: Unrecognized atom type: Fe5+2 (12)


Index:  849  Out of  850


[12:06:59] WARNING: not removing hydrogen atom without neighbors
[12:06:59] WARNING: not removing hydrogen atom without neighbors


List saved to /content/drive/My Drive/kcat_minidatasets/testNew849.pkl


In [ ]:
def make_datasets_train(x, y):

  # Initialize an empty list to hold the processed data
  processed_data = []

  # Process each row in the DataFrame
  for index, row in data_train.iterrows():
      if index > y:
        break
      if index >= x:

        # Process substrates
        print("Index: ", index ," Out of ", len(data_train))
        substrate_graphs = [inchi_to_graph(inchi) for inchi in row['substrates']]
        for item in substrate_graphs:
          if item is None:
            substrate_graphs.remove(item)
        if substrate_graphs:
            aggregated_substrate_graph = aggregate_graphs(substrate_graphs)
        else:
            aggregated_substrate_graph = None

        # Process products
        product_graphs = [inchi_to_graph(inchi) for inchi in row['products']]
        for item in product_graphs:
          if item is None:
            product_graphs.remove(item)
        if product_graphs and substrate_graphs:
          aggregated_product_graph = aggregate_graphs(product_graphs)
        else:
          aggregated_product_graph = None

        # Process protein substrate
        uniprot_id = row['Uniprot IDs']

        pdb_ids = map_uniprot_to_pdb(uniprot_id)
        if pdb_ids:
            pdb_id = pdb_ids[0]
            protein_structure = fetch_protein_structure(pdb_id)
            protein_graph = protein_structure_to_graph(protein_structure)

        else:
            protein_graph = None


        # Ground truth label
        log10_kcat = row['log10_kcat']
        embedding = row['ESM1b']
        fingerprint = row['difference_fp']
        # Append the processed data to the list
        processed_data.append({
            'aggregated_substrate_graph': aggregated_substrate_graph,
            'aggregated_product_graph': aggregated_product_graph,
            'protein_graph': protein_graph,
            'esm_embedding': embedding,
            'log10_kcat': log10_kcat,
            'fingerprint': fingerprint
        })
  # Define the path to save the processed data
  file_path = '/content/drive/My Drive/kcat_minidatasets/trainNew' + str(y) + '.pkl'

  # Save the list to the specified file
  with open(file_path, 'wb') as file:
      pickle.dump(processed_data, file)

  print(f"List saved to {file_path}")

In [ ]:
make_datasets_train(0, 200)
make_datasets_train(200, 400)
make_datasets_train(400, 600)
make_datasets_train(600, 800)
make_datasets_train(800, 1000)
make_datasets_train(1000, 1200)
make_datasets_train(1200, 1400)
make_datasets_train(1400, 1600)
make_datasets_train(1600, 1800)
make_datasets_train(1800, 2000)
make_datasets_train(2000, 2200)
make_datasets_train(2200, 2400)
make_datasets_train(2400, 2600)
make_datasets_train(2600, 2800)
make_datasets_train(2800, 3000)
make_datasets_train(3000, 3200)
make_datasets_train(3200, 3400)



Index:  0  Out of  3421
Index:  1  Out of  3421
Index:  2  Out of  3421


ERROR:root:Error creating residue graph: 'MSE'


Index:  3  Out of  3421
Index:  4  Out of  3421


[12:07:09] WARNING: not removing hydrogen atom without neighbors
[12:07:09] WARNING: not removing hydrogen atom without neighbors


Index:  5  Out of  3421
Index:  6  Out of  3421


...

[14:20:57] UFFTYPER: Unrecognized charge state for atom: 26
[14:20:57] UFFTYPER: Unrecognized charge state for atom: 26


ERROR:root:Error creating residue graph: 'MSE'


Index:  3398  Out of  3421
Index:  3399  Out of  3421


ERROR:root:Error fetching structure for PDB ID none: [Errno 2] No such file or directory: '/tmp/pdbnone.ent'
ERROR:root:Error creating residue graph: 'NoneType' object is not iterable


Desired structure doesn't exist
Index:  3400  Out of  3421
Structure exists: '/tmp/pdb2h2w.ent' 
List saved to /content/drive/My Drive/kcat_minidatasets/trainNew3400.pkl
